In [ ]:
# Importing spark and dependencies
from pyspark.sql import SparkSession

In [ ]:
# Pointing to our Google Project credentials

credentials_location = '/home/Adi/.gc/gcp_service_capstone.json'

In [ ]:
# Initialize Spark Session with GCS config

spark = SparkSession.builder \
    .appName('test') \
    .master('local[*]') \
    .config("spark.jars", "/home/Adi/the-football-pundits/spark-distributed-computing/lib/gcs-connector-hadoop3-2.2.5.jar") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/home/Adi/.gc/gcp_service_capstone.json") \
    .getOrCreate()

In [ ]:
# Read data from GCS
df_appearances = spark.read.parquet('gs://capstone_datalake/raw/appearances.parquet')
df_competitions = spark.read.parquet('gs://capstone_datalake/raw/competitions.parquet')

In [ ]:
# Perform join (assuming no column conflicts)
df_data_all = df_appearances.join(df_competitions, on='competition_id', how='left')

In [ ]:
df_data_all.createOrReplaceTempView('data_all')

In [ ]:

df_data_all.show(10)

In [ ]:
df_data_all.coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('gs://capstone_datalake/processed/data_all.parquet')